In [ ]:
#install necessary dependencies
!pip install flask pyngrok --quiet

#ngrok authentication token
!./ngrok config add-authtoken 2q1QOym4phRHlm9QPGJuJfBY1zo_ye63SKnsh7bsNN6xFMBB

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
#import required libraries
from flask import Flask, request, jsonify
from pyngrok import ngrok
import os

In [ ]:
#initiate the flask app
app = Flask(__name__)

In [ ]:
#file to store user profiles
users_file = "user_profiles.txt"

#initialise the text file if it doesn't exist
if not os.path.exists(users_file):
    open(users_file, 'w').close()

In [ ]:
#root route
@app.route('/')
def index():
    return "Welcome to the User Profile App"

In [ ]:
#helper functions to load, save and update user data
def load_users():
    users = {}
    with open(users_file, 'r') as f:
        for line in f:
            email, age = line.strip().split(',')
            users[email] = {'email': email, 'age': age}
    return users

def save_user(email, age):
    with open(users_file, 'a') as f:
        f.write(f"{email},{age}\n")

def update_users(users):
    with open(users_file, 'w') as f:
        for email, data in users.items():
            f.write(f"{email},{data['age']}\n")

In [ ]:
#routes for user management
@app.route('/add_user', methods=['POST'])
def add_user():
    data = request.get_json()
    email = data.get('email')
    age = data.get('age')
    if not email or not age:
        return jsonify({"error": "Email and age are required."}), 400
    users = load_users()
    if email in users:
        return jsonify({"error": "User already exists."}), 400
    save_user(email, age)
    return jsonify({"message": "User added successfully."}), 201

@app.route('/get_user/<email>', methods=['GET'])
def get_user(email):
    users = load_users()
    if email not in users:
        return jsonify({"error": "User not found."}), 404
    return jsonify(users[email]), 200

@app.route('/update_user/<email>', methods=['PUT'])
def update_user(email):
    data = request.get_json()
    users = load_users()
    if email not in users:
        return jsonify({"error": "User not found."}), 404
    if 'age' in data:
        users[email]['age'] = data['age']
    update_users(users)
    return jsonify({"message": "User updated successfully."}), 200

@app.route('/delete_user/<email>', methods=['DELETE'])
def delete_user(email):
    users = load_users()
    if email not in users:
        return jsonify({"error": "User not found."}), 404
    del users[email]
    update_users(users)
    return jsonify({"message": "User deleted successfully."}), 200

In [ ]:
#starting the flask app
if __name__ == '__main__':
    print("Starting Flask App...")

    #starting ngrok tunnel to expose the flask app publicly
    public_url = ngrok.connect(5000)
    print(f"App is running and publicly accessible at {public_url}")

    #run the flask app
    app.run(port=5000)

Starting Flask App...
App is running and publicly accessible at NgrokTunnel: "https://7917-35-185-65-225.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 08:22:16] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 08:22:17] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 08:24:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 08:32:50] "POST /add_user HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 08:34:22] "POST /add_user HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 08:36:26] "GET /get_user/john@doe.com HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 08:38:47] "PUT /update_user/john@doe.com HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 08:39:51] "DELETE /delete_user/martha@smith.ru HTTP/1.1" 200 -
